<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Quantum_Machine_Learning_(QML).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pennylane

In [ ]:
import torch
import torch.nn as nn
import pennylane as qml

# Define the quantum device
dev = qml.device("default.qubit", wires=1)

# Define the quantum circuit
@qml.qnode(dev)
def quantum_circuit(inputs, weights):
    qml.RX(inputs[0], wires=0)  # Apply RX gate
    qml.RY(weights[0], wires=0)  # Apply RY gate
    return qml.expval(qml.PauliZ(0))  # Return expectation value of Pauli-Z operator

# Define the HybridQNN class
class HybridQNN(nn.Module):
    def __init__(self):
        super(HybridQNN, self).__init__()
        self.fc1 = nn.Linear(1, 1)  # Classical linear layer
        weight_shapes = {"weights": (1,)}
        self.quantum_layer = qml.qnn.TorchLayer(quantum_circuit, weight_shapes)  # Quantum layer

    def forward(self, x):
        x = self.fc1(x)  # Forward pass through the classical layer
        x = x.view(-1)  # Flatten the tensor to match the input shape expected by the quantum layer
        x = self.quantum_layer(x)  # Forward pass through the quantum layer
        return x.view(-1, 1)  # Reshape the tensor to match the expected output shape

# Example usage
model = HybridQNN()
input_data = torch.randn(10, 1)  # Example input (batch_size=10, input_dim=1)
output = model(input_data)

# Print the shape of the output
print(output.shape)  # Expected shape: [10, 1]